**35. ¿Cuál es el mínimo que ha durado desde su registro un usuario bloqueado en la plataforma?**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Descargo y leo el CSV

In [ ]:
downloaded = drive.CreateFile({'id':"1nIzHWsLqzJWq9XuKEweSwMCPEGkysoGB"})
downloaded.GetContentFile('logs.csv') 

In [ ]:
!pip install pyarrow
texts = pd.read_csv('logs.csv')
texts.to_parquet('logs.parquet')
del texts

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('logs.parquet')
rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rdd.take(5)

[Row(item_id=1, timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id=2, timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id=3, timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash'),
 Row(item_id=4, timestamp='2004-12-23T07:55:09Z', contributor_username='Sanbec', contributor_id=7856.0, contributor_ip=None, comment='Sin información de origen ni de 

Primero voy a necesitar solo los usuarios que recibieron algún bloqueo y luego busco el minimo por usuario:

(paso el dato de la fecha a tipo datetime para luego poder trabajar mejor)

VOY A DESCARTAR A ALGUNOS USUARIOS YA QUE TIENEN FECHAS DE BLOQUEO ANTERIORES AL DEL REGISTRO (Damian me dio el ok)

In [ ]:
rdd2 = rdd.filter(lambda x: x.contributor_id != None and x.timestamp != None and x.contributor_id != 2075 and x.contributor_id != 5874 and x.contributor_id != 7856
                  and x.contributor_id != 15707 and x.contributor_id != 20096 and x.contributor_id != 23834 and x.contributor_id != 25063 and x.contributor_id != 32187
                  and x.contributor_id != 34991 and x.contributor_id != 38328 and x.contributor_id != 38723 and x.contributor_id != 39094 and x.contributor_id != 43173)

In [ ]:
bloqueados = rdd2.filter(lambda x: x.logtype == 'block').map(lambda x: (x.contributor_id, pd.to_datetime(x.timestamp))).reduceByKey(lambda a,b: a if a < b else b)

bloqueados.take(5)

[(13770.0, Timestamp('2004-12-23 08:28:32+0000', tz='UTC')),
 (9305.0, Timestamp('2004-12-28 01:38:59+0000', tz='UTC')),
 (625.0, Timestamp('2005-02-05 00:07:35+0000', tz='UTC')),
 (24300.0, Timestamp('2005-05-08 01:39:35+0000', tz='UTC')),
 (6585.0, Timestamp('2005-05-12 23:12:43+0000', tz='UTC'))]

Ahora busco los datos que tengan logs de registro (newusers)

In [ ]:
registros = rdd2.filter(lambda x: x.logtype == 'newusers').map(lambda x: (x.contributor_id, pd.to_datetime(x.timestamp))).reduceByKey(lambda a,b: a if a < b else b)

registros.take(5)

[(53615.0, Timestamp('2005-09-08 02:30:15+0000', tz='UTC')),
 (54040.0, Timestamp('2005-09-10 10:33:43+0000', tz='UTC')),
 (54165.0, Timestamp('2005-09-10 20:33:58+0000', tz='UTC')),
 (54410.0, Timestamp('2005-09-11 22:57:14+0000', tz='UTC')),
 (54690.0, Timestamp('2005-09-12 23:52:44+0000', tz='UTC'))]

Ahora hago un join con los nuevos usuarios y los bloqueados, para quedarme unicamente con usuarios que cuenten con amba información:

La primer fecha indica el primer bloqueo y la segunda el de registro.

In [ ]:
join = bloqueados.join(registros)
join.take(5)

[(2135565.0,
  (Timestamp('2013-06-12 12:07:43+0000', tz='UTC'),
   Timestamp('2012-02-26 19:08:23+0000', tz='UTC'))),
 (245395.0,
  (Timestamp('2007-12-15 06:20:20+0000', tz='UTC'),
   Timestamp('2006-10-27 21:43:58+0000', tz='UTC'))),
 (374995.0,
  (Timestamp('2007-10-10 15:19:39+0000', tz='UTC'),
   Timestamp('2007-04-21 15:56:02+0000', tz='UTC'))),
 (775805.0,
  (Timestamp('2010-04-26 21:52:11+0000', tz='UTC'),
   Timestamp('2008-07-31 19:58:08+0000', tz='UTC'))),
 (108970.0,
  (Timestamp('2007-12-18 19:49:11+0000', tz='UTC'),
   Timestamp('2006-03-10 11:35:13+0000', tz='UTC')))]

Restando estas fechas, puedo obtener la cantidad de tiempo que paso un usuario desde su registro a su primer bloqueo.

In [ ]:
join.map(lambda x: x[1][0] - x[1][1]).takeOrdered(10)

[Timedelta('105 days 00:04:24'),
 Timedelta('120 days 10:28:04'),
 Timedelta('122 days 19:32:49'),
 Timedelta('139 days 13:50:37'),
 Timedelta('140 days 02:34:31'),
 Timedelta('149 days 07:10:22'),
 Timedelta('163 days 21:06:14'),
 Timedelta('165 days 16:23:13'),
 Timedelta('171 days 23:23:37'),
 Timedelta('175 days 23:41:11')]

Entonces el tiempo minimo es de 105 dias